In [1]:
! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.9 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


import os

# Replace the following variables with your OpenAI details
os.environ["OPENAI_API_KEY"] = "REPLACE_YOUR_KEY_HERE"


In [4]:

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI( model="gpt-4o",   temperature=0 )

concatenated_content = """
import argparse
import sys
import time
from kubernetes import client, config, watch
from kubernetes.client.rest import ApiException

# Load kubeconfig (~/.kube/config) or in-cluster config
try:
    config.load_kube_config()
except Exception:
    config.load_incluster_config()

core_v1 = client.CoreV1Api()
apps_v1 = client.AppsV1Api()
batch_v1 = client.BatchV1Api()


def list_pods(namespace=None):
    #List all pods, optionally in a specific namespace
    print(f"Listing pods in namespace: {namespace or 'all'}")
    pods = core_v1.list_pod_for_all_namespaces() if not namespace else core_v1.list_namespaced_pod(namespace)
    for pod in pods.items:
        print(f"{pod.status.pod_ip}\t{pod.metadata.namespace}\t{pod.metadata.name}")


def list_services(namespace=None):
    #List services in a namespace or across all namespaces
    print(f"Listing services in namespace: {namespace or 'all'}")
    svcs = core_v1.list_service_for_all_namespaces() if not namespace else core_v1.list_namespaced_service(namespace)
    for svc in svcs.items:
        print(f"{svc.metadata.namespace}\t{svc.metadata.name}\t{svc.spec.type}\t{svc.spec.cluster_ip}")


def create_namespace(name):
    #Create a new namespace
    ns_body = client.V1Namespace(metadata=client.V1ObjectMeta(name=name))
    try:
        core_v1.create_namespace(ns_body)
        print(f"Namespace '{name}' created.")
    except ApiException as e:
        print(f"Failed to create namespace: {e}")


def delete_namespace(name):
    #Delete a namespace
    try:
        core_v1.delete_namespace(name)
        print(f"Namespace '{name}' deletion initiated.")
    except ApiException as e:
        print(f"Failed to delete namespace: {e}")


def create_deployment(name, image, replicas, namespace='default'):
    #Create a Deployment in the specified namespace
    container = client.V1Container(
        name=name,
        image=image,
        ports=[client.V1ContainerPort(container_port=80)]
    )
    template = client.V1PodTemplateSpec(
        metadata=client.V1ObjectMeta(labels={'app': name}),
        spec=client.V1PodSpec(containers=[container])
    )
    spec = client.V1DeploymentSpec(
        replicas=replicas,
        template=template,
        selector={'matchLabels': {'app': name}}
    )
    deployment = client.V1Deployment(
        metadata=client.V1ObjectMeta(name=name),
        spec=spec
    )
    try:
        apps_v1.create_namespaced_deployment(namespace=namespace, body=deployment)
        print(f"Deployment '{name}' created in namespace '{namespace}'.")
    except ApiException as e:
        print(f"Failed to create deployment: {e}")


def scale_deployment(name, replicas, namespace='default'):
    #Scale an existing Deployment
    body = {'spec': {'replicas': replicas}}
    try:
        apps_v1.patch_namespaced_deployment(name=name, namespace=namespace, body=body)
        print(f"Deployment '{name}' scaled to {replicas} replicas.")
    except ApiException as e:
        print(f"Failed to scale deployment: {e}")


def get_pod_logs(name, namespace='default', tail_lines=100):
    #Fetch logs from a Pod
    try:
        logs = core_v1.read_namespaced_pod_log(name=name, namespace=namespace, tail_lines=tail_lines)
        print(f"Logs from {name} (last {tail_lines} lines):\n{logs}")
    except ApiException as e:
        print(f"Failed to fetch logs: {e}")


def exec_command_in_pod(name, namespace='default', command=['/bin/sh', '-c', 'ls']):
    #Execute a command in a Pod and return the output
    try:
        resp = stream(core_v1.connect_get_namespaced_pod_exec,
                      name, namespace,
                      command=command,
                      stderr=True, stdin=False,
                      stdout=True, tty=False)
        print(f"Command output:\n{resp}")
    except ApiException as e:
        print(f"Exec failed: {e}")


list_pods()
list_services('default')
create_namespace("amdocsdemo1")
create_deployment("httpdemo2", "httpd", 3)
time.sleep(30)
list_pods()
scale_deployment("httpdemo2", 5)
list_pods()

pod_name = "REPLACE_POD_NAME_HERE"
exec_command_in_pod(pod_name, command=['/bin/sh', '-c', 'mkdir /hello'])
get_pod_logs(pod_name)
"""


<ipython-input-4-189a747fd535>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI( model="gpt-4o",   temperature=0 )


In [10]:

# Now start the langchain pydantic

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in Kubernetes Python SDK. \n
    Here is a sample python Kubernetes code:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided sample code. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer as json with a description of the code solution. \n
    Then list the import statements. And finally generate the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

In [11]:

# Data model
class code(BaseModel):
    """Code output"""

    description: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")



In [20]:
import json
# Use .bind(response_format={"type": "json_object"}) instead of with_structured_output
code_gen_chain = code_gen_prompt | llm.bind(response_format={"type": "json_object"})
question = "How to create a deployment with name dep2, 2 replicas and image is nginx?"

solution_json_str = code_gen_chain.invoke({"context":concatenated_content,"messages":[("user",question)]})

# Parse the JSON string response into the Pydantic model
# We need to extract the content from the AIMessage object
solution_data = json.loads(solution_json_str.content)


In [13]:
print(solution_data)


{'description': "To create a deployment named 'dep2' with 2 replicas using the 'nginx' image, you can use the `create_deployment` function from the provided sample code. This function constructs a deployment object with the specified parameters and uses the Kubernetes Python client to create the deployment in the specified namespace. In this example, the deployment will be created in the 'default' namespace.", 'imports': ['from kubernetes import client, config', 'from kubernetes.client.rest import ApiException'], 'code': 'def create_deployment(name, image, replicas, namespace=\'default\'):\n    # Create a Deployment in the specified namespace\n    container = client.V1Container(\n        name=name,\n        image=image,\n        ports=[client.V1ContainerPort(container_port=80)]\n    )\n    template = client.V1PodTemplateSpec(\n        metadata=client.V1ObjectMeta(labels={\'app\': name}),\n        spec=client.V1PodSpec(containers=[container])\n    )\n    spec = client.V1DeploymentSpec(\

In [22]:

print(solution_data['description'])

print(solution_data['imports'])

print(solution_data['code'])

To create a deployment named 'dep2' with 2 replicas using the 'nginx' image, you can use the `create_deployment` function from the provided sample code. This function constructs a deployment object with the specified parameters and uses the Kubernetes Python client to create the deployment in the specified namespace. In this example, the deployment will be created in the 'default' namespace.
['from kubernetes import client, config', 'from kubernetes.client.rest import ApiException']
def create_deployment(name, image, replicas, namespace='default'):
    # Create a Deployment in the specified namespace
    container = client.V1Container(
        name=name,
        image=image,
        ports=[client.V1ContainerPort(container_port=80)]
    )
    template = client.V1PodTemplateSpec(
        metadata=client.V1ObjectMeta(labels={'app': name}),
        spec=client.V1PodSpec(containers=[container])
    )
    spec = client.V1DeploymentSpec(
        replicas=replicas,
        template=template,
 

NotImplementedError: 